In [1]:
import pandas as pd
from sklearn.utils import shuffle


In [2]:
df = pd.read_csv('task2_en_training.tsv', sep='\t')[['class', 'tweet']]
df['class'] = df['class'].apply(lambda x: '__label__' + str(x))

df = shuffle(df)
#df[['class', 'tweet']].to_csv('TRAIN.txt', header=False, index=False, sep="\t")

df_train = df[['class', 'tweet']][:16400]
df_train[['class', 'tweet']].to_csv('TRAIN.txt', header=False, index=False, sep="\t")

df_test= df[['class', 'tweet']][16400:]
df_test[['class', 'tweet']].to_csv('TEST.txt', header=False, index=False, sep="\t")
(df_train, df_test)

(            class                                              tweet
 10945  __label__0  @dralmulaifi @tkmem  sibutramine i think, whic...
 12214  __label__0  anybody know what a buzzy bee is for enbrel sh...
 1985   __label__1  what i lack in money and personality, i make u...
 11857  __label__0  @andystu1212 found this 'you can also buy a sh...
 11423  __label__1  i'm allergic to tramadol, codeine, buprenorphi...
 ...           ...                                                ...
 382    __label__0  “@kylethompson168: working out on vyvanse is s...
 16608  __label__0  "mom, dad... januvia and i broke up. i want yo...
 15527  __label__0  @waaitwhaa cause my adderall hasn't kicked in yet
 277    __label__0  rt @pdrennert: @omicsomicsblog @matthewherper ...
 13179  __label__0  @4wannabes @hustlerhaze it's not even medicati...
 
 [16400 rows x 2 columns],
             class                                              tweet
 13458  __label__0  i forgot to take my nexium last night. ma

In [3]:
import fasttext

graph = []
maxima, maxima_idx = 0, 0

for epoch in range(15):
    model = fasttext.train_supervised(input='TRAIN.txt', epoch=epoch)
    All, Pos = 0, 0
    for (clas, tweet) in zip(df_test['class'], df_test['tweet']):
        All += 1
        if int(clas[-1]) == int(model.predict(tweet, k=2)[0][0][-1]): 
            #print(int(clas[-1]),int(model.predict(tweet, k=2)[0][0][-1]))
            Pos += 1
    graph.append(Pos/All)
    if Pos/All > maxima: maxima, maxima_idx = Pos/All, epoch

print("the best accuracy:")
print(maxima, f'on the {maxima_idx} epoch')


the best accuracy:
0.9276061776061776 on the 10 epoch


In [4]:
graph = pd.DataFrame(graph)
graph.plot(figsize=(15,5))

In [5]:
model.words[:10],[model.get_word_vector(word) for word in model.words][1]

(['</s>', 'the', 'i', 'to', 'a', 'and', 'of', 'for', 'is', 'in'],
 array([ 0.00326706, -0.00430413,  0.01399148, -0.00266695,  0.00795278,
         0.00895169, -0.00680315,  0.00035657,  0.00389308,  0.00477879,
        -0.0037471 , -0.00518909, -0.00321066,  0.00793664, -0.0119857 ,
        -0.00665465,  0.002894  ,  0.00566903, -0.00703102, -0.00392427,
        -0.00126944,  0.01858267,  0.00607132,  0.00041299, -0.00967281,
        -0.0126015 , -0.0032378 , -0.00773381, -0.00756601,  0.00405319,
        -0.00555021,  0.00066837, -0.00669392, -0.01311075, -0.019938  ,
        -0.00029354, -0.00808612, -0.00168603, -0.00343301, -0.00540919,
        -0.00760654,  0.01662572, -0.00161817,  0.01236784,  0.0132695 ,
         0.00117704,  0.01189979, -0.00315631, -0.01253506,  0.00957094,
        -0.01423809,  0.00327584,  0.00946277,  0.00120247, -0.0074295 ,
         0.00619731, -0.00829167, -0.00337309,  0.00340299, -0.00487749,
         0.00327013, -0.00106946, -0.00280348,  0.00202286

In [6]:
import numpy as np
tweets = df_test['tweet'].to_list()

In [7]:
model = fasttext.train_supervised(input='TRAIN.txt', epoch=maxima_idx)
predictions = model.predict(tweets, k=2)

predictions = [int(pred[0][-1]) for pred in predictions[0]]
tests = [int(test[-1]) for test in df_test['class']]

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

print("SVM Accuracy Score -> ", accuracy_score(predictions, tests)*100)
print("SVM F1 Score -> ", f1_score(predictions, tests)*100)
print("SVM Roc_auc Score -> ", roc_auc_score(predictions, tests)*100)

SVM Accuracy Score ->  92.73648648648648
SVM F1 Score ->  48.370497427101206
SVM Roc_auc Score ->  78.69260204081631
